In [51]:
import os
import torch
import sys
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from tqdm import tqdm

def find_question_match(dataset, query):
    """
    Given a dataset and a query string, this function loops through the dataset
    and prints the 'id', 'question', and 'answer' fields for each sample whose 
    question contains the query string (case-insensitive match).
    
    Parameters:
    - dataset (list of dict): A list where each element is a dictionary with keys
                              including 'id', 'question', and 'answer'.
    - query (str): The query string to match against the question field.
    
    Returns:
    - None. The function prints the matching results.
    """
    query_lower = query.lower()
    N=len(dataset)
    for i in range(N):
        sample = dataset[i]
        if query_lower in sample['question'].lower():
            print(f"ID: {sample.get('id')}")
            print(f"Question: {sample.get('question')}")
            print(f"Answer: {sample.get('answer')}")
            print("-" * 80)
            break


def find_question(dataset):
    """
    Given a dataset and a query string, this function loops through the dataset
    and prints the 'id', 'question', and 'answer' fields for each sample whose 
    question contains the query string (case-insensitive match).
    
    Parameters:
    - dataset (list of dict): A list where each element is a dictionary with keys
                              including 'id', 'question', and 'answer'.
    - query (str): The query string to match against the question field.
    
    Returns:
    - None. The function prints the matching results.
    """
    res = {}
    N=len(dataset)
    for i in tqdm(range(N)):
        sample = dataset[i]
        res[sample['question']] = sample['id']
    return res
        

In [79]:
for k in res:
    if "Veteran at War Rally?".lower() in k.lower():
        print(k)
        print (res[k])
        break
    

In [ ]:

res[k]


'WebQTrn-2349_e831da3802943dad506eb1e3fb611847'

In [52]:
res = find_question(testset)


100%|██████████| 27639/27639 [09:54<00:00, 46.46it/s]


In [81]:
find_question_match(testset, "in the film with a character named Veteran at War Rally?")

ID: WebQTrn-962_f0c57985929ee8b823983f6e5f104971
Question: What actor played a kid in the film with a character named Veteran at War Rally?
Answer: ['Michael Connor Humphreys']
--------------------------------------------------------------------------------


In [ ]:
for t in testset:
    print (t['question'])


what does jamaican people speak
what did james k polk do before he was president
who plays ken barlow in coronation street
where is jamarcus russell from
where was george washington carver from
what else did ben franklin invent
who was richard nixon married to
who is governor of ohio 2011
who was vice president after kennedy died
where is the fukushima daiichi nuclear plant located
what countries are part of the uk
what is my timezone in louisiana
what did st augustine do
who is keyshia cole dad
what kind government does egypt have
what town was martin luther king assassinated in
where did edgar allan poe died
what to do today in atlanta with kids
what electorate does anna bligh represent
where did andy murray started playing tennis
what is the australian dollar called
what timezone is sweden
who did cam newton sign with
what county is frederick md in
what highschool did harper lee go to
what timezone is utah in
who did george w. bush run against for the second term
who is niall fergus

KeyboardInterrupt: 

In [5]:
for i in range(100):
    if len(testset[i]['graph'])<100:
        print (i,len(testset[i]['graph']))

4 92
24 99
70 84
87 37
94 39


In [80]:
ds = load_dataset("rmanluo/RoG-cwq")
testset = ds['test']



Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [19]:
a = [(i,train_set[i]['question']) for i in range(200) if 'cleveland' in train_set[i]['question']]
a

[(1, 'what time zone am i in cleveland ohio')]

In [20]:
len(train_set)

250

In [6]:
res = [train_set[i]['q_entity'] for i in range(2000)]

In [11]:
res = np.array([len(i) for i in res])

In [21]:
np.argsort(res>1)[::-1]

array([ 677, 1143,  281, ..., 1332, 1333,    0])

In [1]:
train_set[0]

NameError: name 'train_set' is not defined

In [26]:
# Re-import necessary libraries after execution state reset
import torch
import networkx as nx
import numpy as np
from scipy.sparse import csr_matrix

# Step 1: Create a sample directed graph (nx_graph)
nx_graph = nx.DiGraph()
nx_graph.add_edges_from([
    (0, 1, {'triple_id': 0, 'relation_id': 0}),
    (1, 2, {'triple_id': 1, 'relation_id': 1}),
    (2, 3, {'triple_id': 2, 'relation_id': 2}),
    (3, 4, {'triple_id': 3, 'relation_id': 3}),
    (4, 5, {'triple_id': 4, 'relation_id': 4}),
    (2, 5, {'triple_id': 5, 'relation_id': 5})
])

# Step 2: Convert to a line graph
line_g = nx.line_graph(nx_graph)

# Step 3: Enumerate nodes in the line graph
line_graph_nodes = list(line_g.nodes())  # Each node corresponds to an edge in the original graph
line_node_map = {node: i for i, node in enumerate(line_graph_nodes)}
num_nodes = len(line_graph_nodes)

# Step 4: Build edge index list
line_edge_index_list = []
for (n1, n2) in line_g.edges():
    src_idx = line_node_map[n1]
    dst_idx = line_node_map[n2]
    line_edge_index_list.append([src_idx, dst_idx])

# Step 5: Create adjacency matrix (A_1hop) using SciPy CSR
row_1hop = []
col_1hop = []

for e in line_edge_index_list:
    row_1hop.append(e[0])
    col_1hop.append(e[1])

if num_nodes == 0:
    adj_1hop_csr = csr_matrix((0,0), dtype=np.float32)
else:
    data_1hop = np.ones(len(row_1hop), dtype=np.float32)
    adj_1hop_csr = csr_matrix((data_1hop, (row_1hop, col_1hop)), shape=(num_nodes, num_nodes))

# Step 6: Convert to PyTorch sparse tensor
def csr_to_torch_sparse(csr_mat):
    coo_mat = csr_mat.tocoo()
    indices = np.vstack((coo_mat.row, coo_mat.col))
    values = coo_mat.data
    shape = coo_mat.shape
    sp_tensor = torch.sparse_coo_tensor(
        indices=torch.LongTensor(indices),
        values=torch.FloatTensor(values),
        size=shape
    ).coalesce()
    return sp_tensor

A_1hop = csr_to_torch_sparse(adj_1hop_csr)

# Display the adjacency matrix and PyTorch sparse tensor representation
adj_1hop_dense = adj_1hop_csr.toarray()
A_1hop_indices = A_1hop.indices()
A_1hop_values = A_1hop.values()



In [31]:
A_1hop.indices()

tensor([[0, 1, 1, 2, 4],
        [1, 2, 3, 4, 5]])